In [1]:
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import wordnet as wn
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer
import spacy
import collections
import numpy as np
from spacy.en import English
import string
import unicodedata
import gensim
import re
from collections import Counter
from itertools import chain
from scipy.spatial import distance
import spacy

nlp = spacy.load('en')
parser = English()
sqd_data = pd.read_json("/data/data_repo/train-v1.1.json")

In [8]:
sqd_data.head()
print(len(sqd_data))

442


In [3]:
word2vec_model = gensim.models.KeyedVectors.load_word2vec_format('/home/aleena/Downloads/GoogleNews-vectors-negative300.3.bin', binary=True)


In [4]:
stopword = stopwords.words('english')
qtntype_list = ['How', 'What', 'When', 'Why', 'Which', 'Who','Whom', 'Where']
pun_list = list(string.punctuation)
delelements_list = stopword + qtntype_list + pun_list

In [5]:
qtntype_list

['How', 'What', 'When', 'Why', 'Which', 'Who', 'Whom', 'Where']

In [6]:
row_list = ['How many','How much','What','When','Where','Who', 'Which','Whom','PERSON',
          'NORP', 'FAC','LOC','PRODUCT', 'EVENT','WORK_OF_ART','LANGUAGE' ,'DATE' ,
          'TIME' ,'PERCENT' ,'MONEY' ,'ORDINAL','CARDINAL','GPE', 'ORG']


column_list = ['PERSON','NORP', 'FAC','LOC','PRODUCT', 'EVENT','WORK_OF_ART','LANGUAGE' ,'DATE' ,
               'TIME' ,'PERCENT' ,'MONEY' ,'ORDINAL','CARDINAL','GPE','ORG']

In [7]:
co_occ_matrix = np.zeros((len(row_list), len(column_list)), dtype = int)
co_occ_matrix

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0,

### Co-occurence matrix on 442 paragraphs

In [10]:
def replace_all(repls, str):
   # return re.sub('|'.join(repls.keys()), lambda k: repls[k.group(0)], str)                                    
   return re.sub('|'.join(re.escape(key) for key in repls.keys()),
                 lambda k: repls[k.group(0)], str)

In [11]:
def preprocess_question(para_question):
    question_list = []
    for i in range(len(para_question)):
        l1 = para_question[i]['question']
        l1 = unicodedata.normalize('NFKD', l1).encode('ascii','ignore').decode('utf-8')
        question_list.append(l1)
    
    #Appropriate question type replacement
    Question_list = []
    for i in range(len(question_list)):
        Question_list.append(replace_all({"how": "How", "when": "When","what": "What","whom": "Whom","which": "Which", "where": "Where", "who":"Who" }, question_list[i]))
        
    return Question_list   

In [12]:
def preprocess_text(para_context):

    context = unicodedata.normalize('NFKD', para_context).encode('ascii','ignore').decode('utf-8')
    #print(context)
    #sentence_tokeniztion
    senttokenized_context = sent_tokenize(context)
    sentencelist = []
    for sent_token in senttokenized_context:
        sentencelist.append(sent_token) 
    
    sentence_list = []
    lemmatizer = WordNetLemmatizer()


    for i in sentencelist:
        words = nltk.word_tokenize(i)
        words = ([x.lower() for x in words])
        words = ([x for x in words if x not in delelements_list])
        words = [lemmatizer.lemmatize(x) for x in words]
        words = [[s.encode('ascii').decode('utf-8') for s in words]]
        sentence_list.append(words)
    return sentencelist, sentence_list

In [13]:
def question_type_extractor(question_mapping):
    Question_mapping = []
    for i in range(len(question_mapping)):
        if question_mapping[i] != 'How many' and question_mapping[i] != 'How much':
            Question_mapping.append(question_mapping[i].split()[0])
        else:
             Question_mapping.append(question_mapping[i])
    return Question_mapping

In [14]:
def wmd_que_ans_pair(que_list, sen_list):
    wmd_matrix = np.zeros([len(que_list),len(sen_list)]) 
    for i in range(len(que_list)):
        for j in range(len(sen_list)):
            wmd_matrix[i,j] =  word2vec_model.wmdistance(que_list[i],sen_list[j])  
    wmd_df = pd.DataFrame(wmd_matrix)
    wmd_df['sentence'] = wmd_df.T.idxmin()
    wmd_df['qnno'] = wmd_df.index
    que_no_ans_sen_no = []
    for i in range(len(wmd_df)):
        que_no_ans_sen_no.append([wmd_df['qnno'][i], wmd_df['sentence'][i]])
    qn_sentence = []
    for i in range(len(que_no_ans_sen_no)):
        if que_no_ans_sen_no[i][0] == que_list.index(que_list[i]):
            qn_sentence.append(que_list[i])
    an_sentence = []
    for i in range(len(que_no_ans_sen_no)):
        for j in range(len(sen_list)):
            if que_no_ans_sen_no[i][1] == sen_list.index(sen_list[j]):
                an_sentence.append(sen_list[j]) 
    return qn_sentence, an_sentence

In [15]:
def que_ner_ans_ner(que_sentence, ans_sentence):
    ans_sentence_mapping = []
    for i in range(len(ans_sentence)):
        doc_ans = nlp(ans_sentence[i])
        for ent in doc_ans.ents:
            ans_sentence_mapping.append([i,ent.label_])

    que_sentence_mapping = []
    for i in range(len(que_sentence)):
        doc_que = nlp(que_sentence[i])
        for ent in doc_que.ents:
            que_sentence_mapping.append([i,ent.label_])
    return que_sentence_mapping, ans_sentence_mapping

In [16]:
def que_df_ans_df(que_sen_map, ans_sen_map):
    qno_anstag_df = pd.DataFrame(ans_sen_map,columns = ['QuestionNo','AnswerTag'])
    qno_quetag_df = pd.DataFrame(que_sen_map, columns = ['QuestionNo', 'QuestionTag'])
    return qno_quetag_df, qno_anstag_df 

In [17]:
def que_ans_combo_(questionword, answerword):
    ques_ans_combo = []
    for i in range(len(questionword)):
        for j in range(len(answerword)):
            ques_ans_combo.append([questionword[i],answerword[j]])
    return(ques_ans_combo)

In [18]:
def question_type(process_que_list):
    question_mapping = [] 
    for i in range(len(process_que_list)):
        l2 = nltk.word_tokenize(process_que_list[i])
        #print(len(l2))
        #print(l2)
        missed = l2[-1]        
        if missed == '?':
            for j in range(len(l2)):


                if l2[j] in qtntype_list:
                    combine_list = l2[j]+" "+l2[j+1]
                    question_mapping.append(combine_list)

        else:

             for k in range(len(l2)):#8

                if k < len(l2)-1:  #7        

                    if l2[k] in qtntype_list:
                        combine_list = l2[k]+" "+l2[k+1]
                        question_mapping.append(combine_list) 
                else:

                        if l2[k] in qtntype_list:
                            combine_list = l2[k]+" "+'abc'
                            question_mapping.append(combine_list) 
    return question_mapping

### On to the execution part

In [19]:
for i in range(len(sqd_data)):
    print(i)
    paragraph_pointer = sqd_data.data[i]['paragraphs'] 

    #print(i, " ", len(paragraph_pointer))
    for j in range(len(paragraph_pointer)):#no of paragraphs in a row of the df
        
        processed_sen_list, process_sen_tokenized_list = preprocess_text(paragraph_pointer[j]['context']) 
        processed_que_list = preprocess_question(paragraph_pointer[j]['qas'])
        question_mapping = question_type(processed_que_list)
        que_type_extract = question_type_extractor(question_mapping)
        
        que_sen, ans_sen = wmd_que_ans_pair(processed_que_list, processed_sen_list)

        que_sen_map, ans_sen_map = que_ner_ans_ner(que_sen, ans_sen)
        que_no_quetag_df, que_no_anstag_df = que_df_ans_df(que_sen_map, ans_sen_map)
        
        for i in range(len(que_type_extract)):

            question_words = que_no_quetag_df[que_no_quetag_df['QuestionNo']== i]['QuestionTag'].tolist() + [que_type_extract[i]]
            ans_words = que_no_anstag_df[que_no_anstag_df['QuestionNo']==i]['AnswerTag'].tolist()
            combo = que_ans_combo_(question_words, ans_words)
            
            for i, v1 in enumerate(row_list):
                for j, v2 in enumerate(column_list):
                    for k in combo:
                        if [v1, v2] == k:
                            co_occ_matrix[i, j]+=1

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

### Generating Heatmap for the co-occurence matrix

In [20]:
co_occ_matrix_df = pd.DataFrame(co_occ_matrix, index = row_list, columns= column_list)
co_occ_matrix_df

,PERSON,NORP,FAC,LOC,PRODUCT,EVENT,WORK_OF_ART,LANGUAGE,DATE,TIME,PERCENT,MONEY,ORDINAL,CARDINAL,GPE,ORG
How many,1595,1496,165,422,30,155,174,52,2719,112,330,233,488,5361,3512,3522
How much,319,269,20,84,14,26,18,18,729,24,336,352,81,528,650,684
What,20672,15822,1462,4499,458,1298,2082,822,19312,643,2131,854,3984,12788,29030,32959
When,3168,2244,182,580,60,253,295,97,4704,98,53,91,790,1670,4273,4512
Where,2018,1263,304,577,36,111,152,46,1428,33,38,36,397,968,3536,3078
Who,8370,3539,225,703,92,292,530,101,4188,93,94,105,999,1975,5472,7098
Which,3886,2278,339,821,88,208,354,107,2771,80,149,127,684,1737,4890,5017
Whom,297,131,10,24,1,13,14,5,150,0,4,4,40,61,216,261
PERSON,19801,4338,498,1145,156,399,1082,238,7442,212,200,238,1420,3803,8492,9829
NORP,4714,12553,217,1346,115,343,424,405,4348,93,709,134,855,2854,8436,6013


In [21]:
co_occ_matrix_df['Tags'] = co_occ_matrix_df.index

In [22]:
melt_co_occ_matrix_df = pd.melt(co_occ_matrix_df,id_vars = ['Tags'])
melt_co_occ_matrix_df.head()


import plotly.plotly as py
import plotly.graph_objs as go

trace = go.Heatmap(z=melt_co_occ_matrix_df['value'],
                  x=melt_co_occ_matrix_df['Tags'] ,
                  y=melt_co_occ_matrix_df['variable'])
data=[trace]
py.iplot(data)

### Normalizing the matrix to bring out the patterns

In [23]:
from sklearn.preprocessing import normalize
normed_matrix = normalize(co_occ_matrix, axis=1, norm='l1')    #dividing by sum of row...
normed_matrix

/data/Environment/finite_env/lib/python3.5/site-packages/sklearn/utils/validation.py:429: DataConversionWarning:

Data with input dtype int64 was converted to float64 by the normalize function.



array([[ 0.0783168 ,  0.07345576,  0.00810174,  0.02072081,  0.00147304,
         0.00761072,  0.00854365,  0.00255328,  0.13350683,  0.00549936,
         0.01620348,  0.01144064,  0.0239615 ,  0.26323284,  0.17244427,
         0.17293528],
       [ 0.07683044,  0.06478805,  0.00481696,  0.02023121,  0.00337187,
         0.00626204,  0.00433526,  0.00433526,  0.17557803,  0.00578035,
         0.08092486,  0.08477842,  0.01950867,  0.12716763,  0.15655106,
         0.16473988],
       [ 0.13890979,  0.10631921,  0.00982421,  0.03023196,  0.00307763,
         0.00872218,  0.01399043,  0.0055236 ,  0.12977099,  0.00432077,
         0.0143197 ,  0.00573863,  0.02677131,  0.08593162,  0.19507311,
         0.22147484],
       [ 0.1373212 ,  0.09726918,  0.00788903,  0.02514088,  0.00260078,
         0.01096662,  0.01278717,  0.00420459,  0.20390117,  0.00424794,
         0.00229736,  0.00394452,  0.03424361,  0.07238838,  0.1852189 ,
         0.19557867],
       [ 0.14392697,  0.09007917,  0

In [24]:
normed_matrix_df = pd.DataFrame(normed_matrix, index = row_list, columns= column_list)
normed_matrix_df

,PERSON,NORP,FAC,LOC,PRODUCT,EVENT,WORK_OF_ART,LANGUAGE,DATE,TIME,PERCENT,MONEY,ORDINAL,CARDINAL,GPE,ORG
How many,0.078317,0.073456,0.008102,0.020721,0.001473,0.007611,0.008544,0.002553,0.133507,0.005499,0.016203,0.011441,0.023962,0.263233,0.172444,0.172935
How much,0.076830,0.064788,0.004817,0.020231,0.003372,0.006262,0.004335,0.004335,0.175578,0.005780,0.080925,0.084778,0.019509,0.127168,0.156551,0.164740
What,0.138910,0.106319,0.009824,0.030232,0.003078,0.008722,0.013990,0.005524,0.129771,0.004321,0.014320,0.005739,0.026771,0.085932,0.195073,0.221475
When,0.137321,0.097269,0.007889,0.025141,0.002601,0.010967,0.012787,0.004205,0.203901,0.004248,0.002297,0.003945,0.034244,0.072388,0.185219,0.195579
Where,0.143927,0.090079,0.021682,0.041153,0.002568,0.007917,0.010841,0.003281,0.101847,0.002354,0.002710,0.002568,0.028315,0.069039,0.252193,0.219528
Who,0.247078,0.104469,0.006642,0.020752,0.002716,0.008620,0.015645,0.002981,0.123627,0.002745,0.002775,0.003100,0.029490,0.058301,0.161530,0.209529
Which,0.165109,0.096788,0.014403,0.034883,0.003739,0.008838,0.015041,0.004546,0.117735,0.003399,0.006331,0.005396,0.029062,0.073802,0.207767,0.213163
Whom,0.241267,0.106418,0.008123,0.019496,0.000812,0.010561,0.011373,0.004062,0.121852,0.000000,0.003249,0.003249,0.032494,0.049553,0.175467,0.212023
PERSON,0.333952,0.073162,0.008399,0.019311,0.002631,0.006729,0.018248,0.004014,0.125512,0.003575,0.003373,0.004014,0.023949,0.064139,0.143221,0.165770
NORP,0.108221,0.288184,0.004982,0.030901,0.002640,0.007874,0.009734,0.009298,0.099819,0.002135,0.016277,0.003076,0.019629,0.065520,0.193668,0.138043


### Heatmap of the normed matrix


In [25]:
normed_matrix_df['Tags'] = normed_matrix_df.index

#normalized heatmap
melt_normed_matrix_df = pd.melt(normed_matrix_df,id_vars = ['Tags'])
melt_normed_matrix_df.head()


import plotly.plotly as py
import plotly.graph_objs as go

trace = go.Heatmap(z=melt_normed_matrix_df['value'],
                  x=melt_normed_matrix_df['Tags'] ,
                  y=melt_normed_matrix_df['variable'])
data=[trace]
py.iplot(data)

### new_normed_matrix

In [26]:
new_normed_matrix  = 1 - normed_matrix
new_normed_matrix_df = pd.DataFrame(new_normed_matrix, index = row_list, columns= column_list)
new_normed_matrix_df['Tags'] = new_normed_matrix_df.index

In [27]:
melt_new_normed_matrix_df = pd.melt(new_normed_matrix_df,id_vars = ['Tags'])
melt_new_normed_matrix_df.head()

,Tags,variable,value
0,How many,PERSON,0.921683
1,How much,PERSON,0.923170
2,What,PERSON,0.861090
3,When,PERSON,0.862679
4,Where,PERSON,0.856073


In [28]:
answer_dict = {'LOC': 1,'PERSON': 2,'NORP': 3,'FAC': 4,'PRODUCT': 5,'EVENT': 6,'WORK_OF_ART': 7,'LANGUAGE': 8,'DATE':9,'TIME': 10,'PERCENT': 11,'MONEY': 12,'ORDINAL': 13,'CARDINAL': 14,'GPE': 15, 'ORG': 16}
question_dict = {'LOC': 10,
                 'PERSON': 20,
                 'NORP': 30,
                 'FAC': 40,
                 'PRODUCT': 50,
                 'EVENT':60,
                 'WORK_OF_ART':70,
                 'LANGUAGE':80,
                 'DATE': 90,
                 'TIME': 100,
                 'PERCENT': 110,
                 'MONEY': 120,
                 'ORDINAL':130,
                 'CARDINAL': 140,
                 'GPE': 150,
                 'ORG': 160,
                 'How many': 170,
                 'How much': 180,
                 'What': 190,
                 'When': 200,
                 'Where': 210,
                 'Who': 220,
                 'Which':230,
                 'Whom': 240}

In [29]:
melt_new_normed_matrix_df_1 = melt_new_normed_matrix_df.copy()
melt_new_normed_matrix_df_1["Tags"].replace(question_dict, inplace=True)
melt_new_normed_matrix_df_1['variable'].replace(answer_dict, inplace = True)

In [30]:
melt_new_normed_matrix_df_1

,Tags,variable,value
0,170,2,0.921683
1,180,2,0.923170
2,190,2,0.861090
3,200,2,0.862679
4,210,2,0.856073
5,220,2,0.752922
6,230,2,0.834891
7,240,2,0.758733
8,20,2,0.666048
9,30,2,0.891779


### Surpriselib for latent factor

In [31]:
from surprise import SVDpp
from surprise import Dataset
from surprise import accuracy, Reader
reader_new = Reader(line_format='user item rating', sep='\t', rating_scale=(0,1))

In [35]:
reader_new = Reader(line_format='user item rating', sep='\t', rating_scale=(0,1))
melt_1 = Dataset.load_from_file("/data/new_melt.txt", reader_new)
print(melt_1)

In [36]:
len(melt_1.raw_ratings)

384

In [37]:
import math
from sklearn.metrics import mean_squared_error

In [38]:
#for the new melt_1
reconstruction_error_1 = []
for i in range(20):
    
    algo = SVDpp(n_factors=i)
    
    for trainset, testset in melt_1.folds():
        algo.train(trainset)
        predictions_svdpp = algo.test(testset)
        
    
    reconstruct = np.dot(algo.pu, algo.qi.T)
    rmse = mean_squared_error(new_normed_matrix, reconstruct)**0.5
    reconstruction_error_1.append(rmse)
    print(i, rmse)

0 0.940443108281
1 0.940693810191
2 0.94033213154
3 0.94113723407
4 0.940196614452
5 0.938843329703
6 0.940708540872
7 0.940647564646
8 0.941543157484
9 0.941578682551
10 0.940797732857
11 0.942389243663
12 0.940623953881
13 0.940009325993
14 0.940098625115
15 0.941252192664
16 0.94463265427
17 0.940940554227
18 0.941531678686
19 0.945939774413


In [39]:
#we are taking the tenth factor
algo = SVDpp(n_factors=10)
    
for trainset, testset in melt_1.folds():
    algo.train(trainset)
    predictions_svdpp = algo.test(testset)

In [40]:
reconstruct = np.dot(algo.pu, algo.qi.T) #reconstructed matrix
rmse = mean_squared_error(new_normed_matrix, reconstruct)**0.5  

In [41]:
reconstruct_error = new_normed_matrix - reconstruct
reconstruct_error

array([[ 0.99588578,  0.90961056,  0.9374716 ,  1.00737163,  0.99080252,
         0.98688999,  0.92730733,  0.97665882,  0.83750734,  1.02755158,
         1.00009043,  1.01471764,  1.002228  ,  0.76817967,  0.81585939,
         0.7631705 ],
       [ 0.90323276,  0.96646678,  0.99779099,  0.98377711,  1.00427626,
         0.96650367,  1.00797627,  1.02033885,  0.8001877 ,  1.05094506,
         0.97108101,  0.90784484,  1.01116668,  0.9032056 ,  0.79450411,
         0.7871779 ],
       [ 0.80894451,  0.91413062,  0.99660712,  1.00210713,  0.95151751,
         0.97155762,  1.05540826,  0.98780268,  0.85504018,  1.01717866,
         0.98134069,  0.9596306 ,  0.97908994,  0.8885569 ,  0.80443624,
         0.7685548 ],
       [ 0.85192498,  0.91199133,  0.96583476,  0.96957386,  1.00455   ,
         0.97851474,  0.98226601,  1.00887749,  0.78535803,  0.95737568,
         1.00580989,  1.03283732,  0.99338102,  0.90653642,  0.7742986 ,
         0.79978856],
       [ 0.87630221,  0.89448692,  0

In [42]:
reconstruct_error_df = pd.DataFrame(reconstruct_error,index = row_list,columns = column_list)
reconstruct_error_df

,PERSON,NORP,FAC,LOC,PRODUCT,EVENT,WORK_OF_ART,LANGUAGE,DATE,TIME,PERCENT,MONEY,ORDINAL,CARDINAL,GPE,ORG
How many,0.995886,0.909611,0.937472,1.007372,0.990803,0.986890,0.927307,0.976659,0.837507,1.027552,1.000090,1.014718,1.002228,0.768180,0.815859,0.763170
How much,0.903233,0.966467,0.997791,0.983777,1.004276,0.966504,1.007976,1.020339,0.800188,1.050945,0.971081,0.907845,1.011167,0.903206,0.794504,0.787178
What,0.808945,0.914131,0.996607,1.002107,0.951518,0.971558,1.055408,0.987803,0.855040,1.017179,0.981341,0.959631,0.979090,0.888557,0.804436,0.768555
When,0.851925,0.911991,0.965835,0.969574,1.004550,0.978515,0.982266,1.008877,0.785358,0.957376,1.005810,1.032837,0.993381,0.906536,0.774299,0.799789
Where,0.876302,0.894487,0.951935,0.977998,1.005912,1.015075,0.948796,1.018584,0.882017,1.023561,0.995305,1.025256,0.992660,0.948228,0.734578,0.764538
Who,0.739882,0.902501,1.046235,0.921679,1.037925,1.031592,1.007482,1.052399,0.894220,1.004479,0.982178,1.005632,0.918669,0.928374,0.859212,0.810468
Which,0.813002,0.893136,1.000633,0.974571,1.010944,0.980281,0.984591,0.960529,0.879198,0.969854,0.998621,0.961585,0.989742,0.941997,0.744014,0.803797
Whom,0.770033,0.915168,1.034191,1.037187,0.942747,0.980789,1.037121,0.941261,0.882058,1.045076,0.987916,0.947235,0.946120,0.941632,0.864184,0.794513
PERSON,0.700669,0.865847,0.940724,0.925867,1.043669,1.043445,0.888626,1.039681,0.892258,0.963990,0.954171,1.072836,0.952477,0.944942,0.880119,0.844024
NORP,0.860788,0.657523,0.962357,0.925547,1.041687,1.045491,0.993669,1.054829,0.939634,0.983209,0.924167,0.972085,0.970347,0.931752,0.889004,0.881996


### Error plot

In [43]:

import plotly.graph_objs as go

trace1 = go.Bar(
    x=row_list[0:8],
    y=reconstruct_error_df['PERSON'][0:8],
    name='PERSON'
)
trace2 = go.Bar(
    x=row_list[0:8],
    y= reconstruct_error_df['NORP'][0:8],
    name='NORP'
)
trace3 = go.Bar(
    x=row_list[0:8],
    y=reconstruct_error_df['FAC'][0:8],
    name='FAC'
)
trace4 = go.Bar(
    x=row_list[0:8],
    y= reconstruct_error_df['LOC'][0:8],
    name='LOC'
)


trace5 = go.Bar(
    x=row_list[0:8],
    y=reconstruct_error_df['PRODUCT'][0:8],
    name='PRODUCT'
)
trace6 = go.Bar(
    x=row_list[0:8],
    y=reconstruct_error_df['EVENT'][0:8],
    name='EVENT'
)


trace7= go.Bar(
    x=row_list[0:8],
    y=reconstruct_error_df['WORK_OF_ART'][0:8],
    name='WORK_OF_ART'
)
trace8 = go.Bar(
    x=row_list[0:8],
    y= reconstruct_error_df['LANGUAGE'][0:8],
    name='LANGUAGE'
)



trace9 = go.Bar(
    x=row_list[0:8],
    y= reconstruct_error_df['DATE'][0:8],
    name='DATE'
)
trace10= go.Bar(
    x=row_list[0:8],
    y=reconstruct_error_df['TIME'][0:8],
    name='TIME'
)
trace11 = go.Bar(
    x=row_list[0:8],
    y= reconstruct_error_df['PERCENT'][0:8],
    name='PERCENT'
)



trace12 = go.Bar(
    x=row_list[0:8],
    y= reconstruct_error_df['MONEY'][0:8],
    name='MONEY'
)
trace13= go.Bar(
    x=row_list[0:8],
    y=reconstruct_error_df['ORDINAL'][0:8],
    name='ORDINAL'
)
trace14 = go.Bar(
    x=row_list[0:8],
    y= reconstruct_error_df['CARDINAL'][0:8],
    name='CARDINAL'
)


trace15= go.Bar(
    x=row_list[0:8],
    y=reconstruct_error_df['GPE'][0:8],
    name='GPE'
)
trace16 = go.Bar(
    x=row_list[0:8],
    y= reconstruct_error_df['ORG'][0:8],
    name='ORG'
)



data = [trace1, trace2,trace3,trace4,trace5, trace6,trace7,trace8,trace9, trace10,trace11,
        trace12,trace13,trace14,trace15,trace16]
layout = go.Layout(
    barmode='group'
) 



In [44]:
trace1 = go.Bar(
    x=row_list[8:16],
    y=reconstruct_error_df['PERSON'][8:16],
    name='PERSON'
)
trace2 = go.Bar(
    x=row_list[8:16],
    y= reconstruct_error_df['NORP'][8:16],
    name='NORP'
)
trace3 = go.Bar(
    x=row_list[8:16],
    y=reconstruct_error_df['FAC'][8:16],
    name='FAC'
)
trace4 = go.Bar(
    x=row_list[8:16],
    y= reconstruct_error_df['LOC'][8:16],
    name='LOC'
)


trace5 = go.Bar(
    x=row_list[8:16],
    y=reconstruct_error_df['PRODUCT'][8:16],
    name='PRODUCT'
)
trace6 = go.Bar(
    x=row_list[8:16],
    y=reconstruct_error_df['EVENT'][8:16],
    name='EVENT'
)


trace7= go.Bar(
    x=row_list[8:16],
    y=reconstruct_error_df['WORK_OF_ART'][8:16],
    name='WORK_OF_ART'
)
trace8 = go.Bar(
    x=row_list[8:16],
    y= reconstruct_error_df['LANGUAGE'][8:16],
    name='LANGUAGE'
)



trace9 = go.Bar(
    x=row_list[8:16],
    y= reconstruct_error_df['DATE'][8:16],
    name='DATE'
)
trace10= go.Bar(
    x=row_list[8:16],
    y=reconstruct_error_df['TIME'][8:16],
    name='TIME'
)
trace11 = go.Bar(
    x=row_list[8:16],
    y= reconstruct_error_df['PERCENT'][8:16],
    name='PERCENT'
)



trace12 = go.Bar(
    x=row_list[8:16],
    y= reconstruct_error_df['MONEY'][8:16],
    name='MONEY'
)
trace13= go.Bar(
    x=row_list[8:16],
    y=reconstruct_error_df['ORDINAL'][8:16],
    name='ORDINAL'
)
trace14 = go.Bar(
    x=row_list[8:16],
    y= reconstruct_error_df['CARDINAL'][8:16],
    name='CARDINAL'
)


trace15= go.Bar(
    x=row_list[8:16],
    y=reconstruct_error_df['GPE'][8:16],
    name='GPE'
)
trace16 = go.Bar(
    x=row_list[8:16],
    y= reconstruct_error_df['ORG'][8:16],
    name='ORG'
)



data1 = [trace1, trace2,trace3,trace4,trace5, trace6,trace7,trace8,trace9, trace10,trace11,
        trace12,trace13,trace14,trace15,trace16]
layout = go.Layout(
         barmode='group'
) 




In [45]:
trace1 = go.Bar(
    x=row_list[16:24],
    y=reconstruct_error_df['PERSON'][16:24],
    name='PERSON'
)
trace2 = go.Bar(
    x=row_list[16:24],
    y= reconstruct_error_df['NORP'][16:24],
    name='NORP'
)
trace3 = go.Bar(
    x=row_list[16:24],
    y=reconstruct_error_df['FAC'][16:24],
    name='FAC'
)
trace4 = go.Bar(
    x=row_list[16:24],
    y= reconstruct_error_df['LOC'][16:24],
    name='LOC'
)


trace5 = go.Bar(
    x=row_list[16:24],
    y=reconstruct_error_df['PRODUCT'][16:24],
    name='PRODUCT'
)
trace6 = go.Bar(
    x=row_list[16:24],
    y= reconstruct_error_df['EVENT'][16:24],
    name='EVENT'
)


trace7= go.Bar(
    x=row_list[16:24],
    y=reconstruct_error_df['WORK_OF_ART'][16:24],
    name='WORK_OF_ART'
)
trace8 = go.Bar(
    x=row_list[16:24],
    y= reconstruct_error_df['LANGUAGE'][16:24],
    name='LANGUAGE'
)



trace9 = go.Bar(
    x=row_list[16:24],
    y= reconstruct_error_df['DATE'][16:24],
    name='DATE'
)
trace10= go.Bar(
    x=row_list[16:24],
    y=reconstruct_error_df['TIME'][16:24],
    name='TIME'
)
trace11 = go.Bar(
    x=row_list[16:24],
    y= reconstruct_error_df['PERCENT'][16:24],
    name='PERCENT'
)



trace12 = go.Bar(
    x=row_list[16:24],
    y= reconstruct_error_df['MONEY'][16:24],
    name='MONEY'
)
trace13= go.Bar(
    x=row_list[16:24],
    y=reconstruct_error_df['ORDINAL'][16:24],
    name='ORDINAL'
)
trace14 = go.Bar(
    x=row_list[16:24],
    y=reconstruct_error_df['CARDINAL'][16:24],
    name='CARDINAL'
)


trace15= go.Bar(
    x=row_list[16:24],
    y=reconstruct_error_df['GPE'][16:24],
    name='GPE'
)
trace16 = go.Bar(
    x=row_list[16:24],
    y= reconstruct_error_df['ORG'][16:24],
    name='ORG'
)



data2 = [trace1, trace2,trace3,trace4,trace5, trace6,trace7,trace8,trace9, trace10,trace11,
        trace12,trace13,trace14,trace15,trace16]
layout = go.Layout(
    barmode='group'
) 



In [46]:
#error distribution
fig = go.Figure(data=data, layout=layout)
py.iplot(fig)

In [47]:
fig = go.Figure(data=data1, layout=layout)
py.iplot(fig)

In [48]:
fig = go.Figure(data=data2, layout=layout)
py.iplot(fig)

### Euclidean matrix

In [49]:
row_column_list =['How many',
 'How much',
 'What',
 'When',
 'Where',
 'Who',
 'Which',
 'Whom',
 'q_PERSON',
 'q_NORP',
 'q_FAC',
 'q_LOC',
 'q_PRODUCT',
 'q_EVENT',
 'q_WORK_OF_ART',
 'q_LANGUAGE',
 'q_DATE',
 'q_TIME',
 'q_PERCENT',
 'q_MONEY',
 'q_ORDINAL',
 'q_CARDINAL',
 'q_GPE',
 'q_ORG',
 'a_PERSON',
 'a_NORP',
 'a_FAC',
 'a_LOC',
 'a_PRODUCT',
 'a_EVENT',
 'a_WORK_OF_ART',
 'a_LANGUAGE',
 'a_DATE',
 'a_TIME',
 'a_PERCENT',
 'a_MONEY',
 'a_ORDINAL',
 'a_CARDINAL',
 'a_GPE',
 'a_ORG']

In [50]:
qn_ans_list = list(algo.pu) + list(algo.qi)
qn_ans_list

qn_ans_list_copy = qn_ans_list.copy()
len(qn_ans_list_copy)

40

In [51]:
question_answer_dis = np.zeros((len(row_column_list), len(row_column_list))) 

for i in range(len(qn_ans_list)):
    for j in range(len(qn_ans_list_copy)):
        question_answer_dis[i,j] = distance.euclidean(qn_ans_list[i],qn_ans_list_copy[j])


In [52]:
eucli_matrix_df = pd.DataFrame(question_answer_dis, index = row_column_list, columns= row_column_list)
eucli_matrix_df

,How many,How much,What,When,Where,Who,Which,Whom,q_PERSON,q_NORP,...,a_WORK_OF_ART,a_LANGUAGE,a_DATE,a_TIME,a_PERCENT,a_MONEY,a_ORDINAL,a_CARDINAL,a_GPE,a_ORG
How many,0.000000,0.561879,0.616371,0.487149,0.404573,0.766450,0.584588,0.548094,0.582031,0.737504,...,0.406354,0.486660,0.482891,0.594339,0.522378,0.585790,0.540475,0.542535,0.584427,0.350527
How much,0.561879,0.000000,0.480878,0.448108,0.379903,0.598309,0.437290,0.678691,0.604665,0.641615,...,0.535539,0.544536,0.459831,0.607738,0.559518,0.494810,0.519464,0.510904,0.485608,0.351070
What,0.616371,0.480878,0.000000,0.442344,0.406393,0.576519,0.434341,0.362856,0.667555,0.562130,...,0.602800,0.442963,0.438095,0.511161,0.403437,0.390796,0.427325,0.334135,0.543246,0.402326
When,0.487149,0.448108,0.442344,0.000000,0.346512,0.457018,0.322153,0.542797,0.417444,0.533662,...,0.365294,0.393053,0.345812,0.245554,0.326217,0.462935,0.380652,0.198195,0.365916,0.302165
Where,0.404573,0.379903,0.406393,0.346512,0.000000,0.552501,0.369875,0.493419,0.439528,0.461000,...,0.291175,0.440717,0.361899,0.459406,0.329255,0.467512,0.392138,0.371673,0.458888,0.301458
Who,0.766450,0.598309,0.576519,0.457018,0.552501,0.000000,0.568753,0.652528,0.430962,0.565435,...,0.565856,0.608023,0.554408,0.531135,0.436710,0.537414,0.341424,0.430566,0.622053,0.520738
Which,0.584588,0.437290,0.434341,0.322153,0.369875,0.568753,0.000000,0.553751,0.560310,0.488440,...,0.428471,0.315386,0.419432,0.352910,0.375629,0.340144,0.410178,0.392453,0.399265,0.418902
Whom,0.548094,0.678691,0.362856,0.542797,0.493419,0.652528,0.553751,0.000000,0.720372,0.680140,...,0.609540,0.387542,0.529249,0.598681,0.451389,0.415851,0.422010,0.441611,0.652211,0.494832
q_PERSON,0.582031,0.604665,0.667555,0.417444,0.439528,0.430962,0.560310,0.720372,0.000000,0.559737,...,0.281421,0.581232,0.546555,0.440788,0.356939,0.645415,0.405725,0.470860,0.619398,0.492212
q_NORP,0.737504,0.641615,0.562130,0.533662,0.461000,0.565435,0.488440,0.680140,0.559737,0.000000,...,0.569506,0.656338,0.627831,0.530976,0.381170,0.515040,0.493758,0.500176,0.744583,0.561112


### WMD score

In [53]:
#working on a sample 
from gensim.corpora import Dictionary
docu1 = "who is president of US?"
docu2 = "Obama is the president of US."

d1 = docu1.split()
d2 = docu2.split()

In [54]:
question_list = []
l1 = unicodedata.normalize('NFKD', docu1).encode('ascii','ignore').decode('utf-8')
question_list.append(l1)
print(question_list)    
    #Appropriate question type replacement
Question_list = []
for i in range(len(question_list)):
    Question_list.append(replace_all({"how": "How", "when": "When","what": "What","whom": "Whom","which": "Which", "where": "Where" , "who":'Who'}, question_list[i]))
print(Question_list) 


['who is president of US?']
['Who is president of US?']


In [55]:
que_type  = question_type(Question_list)
que_type_clean = question_type_extractor(que_type)

print(que_type)
print(que_type_clean)

['Who is']
['Who']


In [68]:
#extracting question type and NER from the question statement
# map_que_ner = []
# for i in range(len(d1)):
map_que_ner = []
que_ner = nlp(Question_list[0])
for ent in que_ner.ents:
    map_que_ner.append('q_' + ent.label_)

map_que_ner.append(que_type_clean[0])
print(map_que_ner)

['q_GPE', 'Who']


In [70]:
#extracting the answer type from the answer statement
# map_ans_ner = []
# for i in range(len(d2)):
map_ans_ner = []
ans_ner = nlp(docu2)
for ent in ans_ner.ents:
    map_ans_ner.append('a_' + ent.label_)
print(map_ans_ner)

['a_GPE']


In [71]:
new_ner = []
new_ner =  map_que_ner + map_ans_ner
print(new_ner)

['q_GPE', 'Who', 'a_GPE']


In [72]:
unique_removed_new_ner = list(set(new_ner))
unique_removed_new_ner

['Who', 'a_GPE', 'q_GPE']

In [73]:
distance_matrix = np.zeros((len(unique_removed_new_ner), len(unique_removed_new_ner)))
distance_matrix

array([[ 0.,  0.,  0.],
       [ 0.,  0.,  0.],
       [ 0.,  0.,  0.]])

In [79]:
def word_count(doc_ner_list):
    my_count = {}
    for element in doc_ner_list:
        try: my_count[doc_ner_list.index(element)] += 1
        except KeyError: my_count[doc_ner_list.index(element)] = 1
    a = [(k,v) for k,v in my_count.items()]
    return a

def bag_of_words(doc_ner_list, new_ner):
    dis_matrix = np.zeros(len(new_ner), dtype = float)
    nbow = word_count(doc_ner_list)  # Word frequencies.
    doc_len = len(doc_ner_list)
    for idx, freq in nbow:
        dis_matrix[idx] = freq / float(doc_len)  # Normalized word frequencies.
    return dis_matrix

d_1 = bag_of_words(map_que_ner, unique_removed_new_ner)
print(d_1)
d_2 = bag_of_words(map_ans_ner, unique_removed_new_ner)
print(d_2)

[ 0.5  0.5  0. ]
[ 1.  0.  0.]


In [80]:
#euclidean distance extraction
sub_matrix_df = eucli_matrix_df.loc[(new_ner),(new_ner)]
print(sub_matrix_df)
sub_matrix= sub_matrix_df.as_matrix()
print(sub_matrix)

         q_GPE       Who     a_GPE
q_GPE  0.00000  0.673350  0.695830
Who    0.67335  0.000000  0.622053
a_GPE  0.69583  0.622053  0.000000
[[ 0.          0.67334991  0.6958298 ]
 [ 0.67334991  0.          0.62205287]
 [ 0.6958298   0.62205287  0.        ]]


In [81]:
from pyemd import emd
emd(d_1, d_2, sub_matrix)

0.33667481216240913

In [85]:
#computing the actual gensimwmd distance
word2vec_model.init_sims(replace=True)

In [86]:
word2vec_model.wmdistance(docu1.split(), docu2.split())

0.775104919742701